<a href="https://colab.research.google.com/github/Malcuthrad/FlightDelayHackathonOne/blob/main/Logistic_Regression_ANTONIO.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Predicción de Retrasos en Vuelos
## Este proyecto implementa un modelo de Regresión Logística para predecir si un vuelo llegará con más de 15 minutos de retraso, utilizando técnicas de balanceo de datos y preprocesamiento avanzado.

# 1. Instalación y Carga de Librerías
## Primero, instalamos las librerías necesarias (especialmente `feature-engine` que no viene por defecto en Colab) e importamos los módulos de procesamiento y visualización.

In [1]:
# Instalación de feature-engine
!pip install feature_engine

In [9]:
# Librerías
import pandas as pd
import numpy as np
import warnings
from sklearn.model_selection import train_test_split
from feature_engine.selection import DropConstantFeatures
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.preprocessing import StandardScaler

warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', None)

# 2. Carga y Limpieza de Datos
## Cargamos el dataset y realizamos una limpieza inicial, eliminando columnas irrelevantes y valores nulos.

In [10]:
# Carga del dataset
try:
    df = pd.read_csv('vuelos_etl_limpio.csv')
    print("Dataset loaded successfully!")
except FileNotFoundError:
    print("Error: Ensure 'vuelos_etl_limpio.csv' is in the same folder as this script.")

Dataset loaded successfully!


In [11]:
# Conversión de fecha y limpieza de columnas no predictivas
df['FL_DATE'] = pd.to_datetime(df['FL_DATE'])
df_clean = df.drop(columns=['FL_DATE', 'AIRLINE', 'ORIGIN_CITY', 'DEST_CITY',
                            'FL_NUMBER', 'CANCELLED']).dropna()

print(f"Forma del dataset tras limpieza: {df_clean.shape}")

Forma del dataset tras limpieza: (2920056, 20)


# 3. Definición del Target y Balanceo
## Para evitar que el modelo ignore los retrasos debido al desbalance natural (hay más vuelos a tiempo que retrasados), aplicamos Submuestreo (Downsampling) para tener una proporción 50/50.

In [6]:
# Definimos el Target: 1 si el retraso de llegada es > 15 min, 0 de lo contrario
df_clean['DELAYED'] = (df_clean['ARR_DELAY'] > 15).astype(int)

In [7]:
# BALANCING (Downsampling)

# Separación de clases
df_delayed = df_clean[df_clean['DELAYED'] == 1]
df_ontime = df_clean[df_clean['DELAYED'] == 0]

# Reducción de la clase mayoritaria (On-time) para igualar a la minoritaria
df_ontime_balanced = df_ontime.sample(n=len(df_delayed), random_state=42)

# Combinación y mezcla (shuffle)
df_balanced = pd.concat([df_delayed, df_ontime_balanced]).sample(frac=1, random_state=42)

print("Distribución tras el balanceo:")
print(df_balanced['DELAYED'].value_counts())

# 4. Ingeniería de Variables (Feature Engineering)
## Convertimos los formatos de hora (HH:MM:SS) a minutos totales desde la medianoche para que el modelo pueda procesarlos matemáticamente.

In [8]:
def time_to_total_minutes(time_str):
    if pd.isna(time_str) or time_str == '': return 0
    try:
        h, m, s = map(int, str(time_str).split(':'))
        return h * 60 + m
    except: return 0

# Columnas de tiempo a convertir
time_cols = ['CRS_DEP_TIME', 'DEP_TIME', 'WHEELS_OFF', 'WHEELS_ON', 'CRS_ARR_TIME', 'ARR_TIME']
for col in time_cols:
    if col in df_balanced.columns:
        df_balanced[col] = df_balanced[col].apply(time_to_total_minutes)

# 5. Preparación de X e y
## Eliminamos las columnas de "fuga de datos" (data leakage) como `ARR_DELAY` o `DEP_DELAY`, ya que no se conocen antes de que ocurra el vuelo.

In [10]:
# Eliminación de columnas de fuga y variables redundantes
cols_to_drop = ['ARR_DELAY', 'DEP_DELAY', 'YEAR', 'MONTH', 'DAY', 'DEP_TIME','ARR_TIME', 'WHEELS_OFF', 'WHEELS_ON']
X = df_balanced.drop(columns=cols_to_drop + ['DELAYED'])
y = df_balanced['DELAYED']

In [11]:
# Codificación de variables categóricas (One-Hot Encoding)
X = pd.get_dummies(X, columns=['AIRLINE_CODE', 'ORIGIN', 'DEST'], drop_first=True)

# 6. División, Escalado y Entrenamiento
## Escalamos los datos para que variables como `DISTANCE` (grandes) no opaquen a las pequeñas, optimizando la Regresión Logística.

In [12]:
# División en entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
# En este paso se cae colab por falta de RAM

# Escalado de variables (Crucial para modelos lineales)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [ ]:
# Entrenamiento del modelo
log_reg = LogisticRegression(max_iter=1000)
log_reg.fit(X_train_scaled, y_train)

# 7. Evaluación de Resultados
## Generamos la Matriz de Confusión y el Reporte de Clasificación para medir la eficacia.

In [ ]:
# 8. EVALUACIÓN
y_pred = log_reg.predict(X_test_scaled)
print("\nConfusion Matrix:")
print(confusion_matrix(y_test, y_pred))
print("\nClassification Report:")
print(classification_report(y_test, y_pred))

# Resultados obtenidos corriendo el codigo en Visual Studio

## 1. Confusion Matrix

* **Class 0:** On Time
* **Class 1:** Delayed (Arrival Delay > 15 min)

| | Predicted: **On Time** (0) | Predicted: **Delayed** (1) |
| :--- | :---: | :---: |
| **Actual: On Time** (0) | **73,509** (True Negative) | **29,286** (False Positive) |
| **Actual: Delayed** (1) | **37,496** (False Negative) | **65,825** (True Positive) |


## 2. Classification Report

| Class | Precision | Recall | F1-Score | Support |
| :--- | :--- | :--- | :--- | :--- |
| **0 (On Time)** | 0.66 | 0.72 | 0.69 | 102,795 |
| **1 (Delayed)** | 0.69 | 0.64 | 0.66 | 103,321 |
| | | | | |
| **Accuracy** | | | **0.68** | **206,116** |
| **Macro Avg** | 0.68 | 0.68 | 0.68 | 206,116 |
| **Weighted Avg** | 0.68 | 0.68 | 0.68 | 206,116 |

# 9. Guardar el Modelo
## Guardamos el modelo y el escalador para usarlos en el futuro sin necesidad de volver a entrenar.

In [ ]:
# Guardado de archivos .pkl
import joblib

joblib.dump(log_reg, 'flight_delay_model.pkl')
joblib.dump(scaler, 'flight_scaler.pkl')
print("¡Modelo y Escalador guardados correctamente!")

In [ ]:
# Código para descargar los archivos .pkl
from google.colab import files

# Descargar el modelo
files.download('flight_delay_model.pkl')

# Descargar el escalador
files.download('flight_scaler.pkl')

# Descargar el features
files.download('model_features.pkl')
